In [1]:
!pip install groq
import groq

print(groq.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 4.3 MB/s eta 0:00:00
1.0.0


In [2]:
!pip install groq

from google.colab import userdata
from groq import Groq

# API Anahtarını alıyoruz
# (Sol menüdeki "Anahtar" simgesinden GROQ_API_KEY eklemiş olmalısın)
try:
    groq_api_key = userdata.get('GROQ_API_KEY')
    client = Groq(api_key=groq_api_key)

    # Bağlantı testi yapıyoruz
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": "Hello, how are you?",
            }
        ],
        model="llama-3.3-70b-versatile"
    )

    print("✅ BAŞARILI! Modelden gelen cevap:")
    print(chat_completion.choices[0].message.content)

except Exception as e:
    print("❌ HATA OLUŞTU!")
    print(e)
    print("\nLütfen sol menüdeki 'Anahtar' (Secrets) kısmına 'GROQ_API_KEY' eklediğinden emin ol.")

✅ BAŞARILI! Modelden gelen cevap:
Hello. I'm just a language model, so I don't have emotions or feelings like humans do, but I'm functioning properly and ready to assist you. How can I help you today?


In [3]:
# Gerekli Kütüphaneler: ChromaDB (Vektör DB), Sentence-Transformers (Embedding), Wikipedia
!pip install -q chromadb sentence-transformers wikipedia
print("✅ ChromaDB ve gerekli kütüphaneler kuruldu.")

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
# =============================================================================
# 2. ADIM: CHROMADB VEKTÖR VERİTABANI (OTOMATİK TXT OKUMA)
# =============================================================================

import chromadb
from chromadb.utils import embedding_functions
import uuid
import os
import glob # Dosya listelemek için

# --- A. VERİ SETİNİ HAZIRLAMA (KLASÖR TARAMA) ---
print("🌍 Veri Seti Hazırlanıyor (Klasördeki .txt dosyaları taranıyor)...")

full_knowledge_base = []

# Klasördeki tüm .txt uzantılı dosyaları bul
txt_files = glob.glob("*.txt")

# requirements.txt dosyasını hariç tutalım (o kütüphane dosyası, veri değil)
txt_files = [f for f in txt_files if "requirements.txt" not in f]

if not txt_files:
    print("❌ HATA: Hiç .txt dosyası bulunamadı! Lütfen dosyaları sol panele yüklediğinden emin ol.")
else:
    for f_name in txt_files:
        try:
            with open(f_name, "r", encoding="utf-8") as f:
                content = f.read()

                # Dosya ismine göre kaynak etiketi belirle
                if "wiki_" in f_name:
                    source_label = f"SOURCE: Wikipedia File ({f_name})"
                else:
                    source_label = f"SOURCE: Internal Guide ({f_name})"

                full_knowledge_base.append(f"{source_label}\n{content}")
                print(f"   ✅ Dosya İşlendi: {f_name} ({len(content)} karakter)")
        except Exception as e:
            print(f"   ⚠️ Okuma Hatası ({f_name}): {e}")

print(f"📚 Toplam Döküman Kaynağı: {len(full_knowledge_base)}")


# --- B. CHROMADB KURULUMU ---
print("\n💾 ChromaDB Başlatılıyor...")

chroma_client = chromadb.Client()

# Embedding Modeli (Sentence Transformers)
emb_fn = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

# Koleksiyonu Sıfırla ve Yeniden Oluştur
try:
    chroma_client.delete_collection(name="fashion_knowledge")
except:
    pass

collection = chroma_client.create_collection(
    name="fashion_knowledge",
    embedding_function=emb_fn
)


# --- C. VERİLERİ PARÇALA VE KAYDET (CHUNK & EMBED) ---
print("⚙️ Veriler Vektörleştiriliyor ve Kaydediliyor...")

ids = []
documents = []
metadatas = []

for doc in full_knowledge_base:
    # Paragraflara böl (Çift satır başı)
    chunks = [c.strip() for c in doc.split('\n\n') if len(c.strip()) > 50]

    # Kaynak bilgisini ilk satırdan al
    source_line = doc.split('\n')[0]

    for chunk in chunks:
        documents.append(chunk)
        ids.append(str(uuid.uuid4()))
        metadatas.append({"source": source_line})

if documents:
    collection.add(
        documents=documents,
        ids=ids,
        metadatas=metadatas
    )
    print(f"✅ İŞLEM TAMAM: {len(documents)} adet bilgi parçacığı (chunk) veritabanına kaydedildi!")
else:
    print("❌ HATA: Kaydedilecek veri parçası bulunamadı.")

🌍 Veri Seti Hazırlanıyor (Klasördeki .txt dosyaları taranıyor)...
   ✅ Dosya İşlendi: color_theory_analysis.txt (3425 karakter)
   ✅ Dosya İşlendi: wiki_Color_theory.txt (23841 karakter)
   ✅ Dosya İşlendi: body_type_guide.txt (5157 karakter)
   ✅ Dosya İşlendi: fabric_and_care_guide.txt (3108 karakter)
   ✅ Dosya İşlendi: wiki_Fashion_design.txt (27402 karakter)
   ✅ Dosya İşlendi: outfit_rules_and_etiquette.txt (3421 karakter)
   ✅ Dosya İşlendi: wiki_Textile.txt (41475 karakter)
   ✅ Dosya İşlendi: wiki_Clothing_material.txt (3901 karakter)
📚 Toplam Döküman Kaynağı: 8

💾 ChromaDB Başlatılıyor...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

⚙️ Veriler Vektörleştiriliyor ve Kaydediliyor...
✅ İŞLEM TAMAM: 153 adet bilgi parçacığı (chunk) veritabanına kaydedildi!


In [5]:
# 3. ADIM: ARAÇLAR (TOOLS)
# =============================================================================
# 1. HESAP MAKİNESİ (Değişmedi)
def calculator(expression: str) -> str:
    """
    Calculates mathematical expressions.
    Useful for unit conversions (e.g. inch to cm) or budget calculations.
    """
    try:
        allowed_chars = set("0123456789+-*/(). ")
        if not all(c in allowed_chars for c in expression):
            return "Error: Invalid characters."
        return str(eval(expression))
    except Exception as e:
        return f"Error: {e}"

# 2. YENİ RAG ARACI (CHROMADB İLE BAĞLANTILI)
def search_knowledge_base(query: str) -> str:
    """
    Use this tool to find stylistic rules, fabric details, and fashion theory.
    Searches the Vector Database (ChromaDB) for semantic matches from Guides and Wikipedia.
    """
    print(f"\n🔎 [CHROMADB ARAMASI]: '{query}'")

    try:
        # ChromaDB'den en alakalı 3 sonucu istiyoruz
        results = collection.query(
            query_texts=[query],
            n_results=3
        )

        # Sonuç var mı kontrol et
        if not results['documents'] or not results['documents'][0]:
            return "Veritabanında ilgili bilgi bulunamadı."

        # Gelen veriyi formatla
        found_texts = results['documents'][0]
        found_sources = results['metadatas'][0]

        output = "📚 Vektör Veritabanından Bulunanlar:\n"
        for i, text in enumerate(found_texts):
            source = found_sources[i]['source']
            output += f"---\n[Kaynak: {source}]\n{text}\n"

        return output
    except Exception as e:
        return f"Veritabanı hatası: {e}"

# 3. ÜRÜN ARAMA ARACI (CSV - Değişmedi)
# (Bu fonksiyonun çalışması için styles.csv yüklü olmalı ve df tanımlı olmalı)
# Eğer df hatası alırsan yukarıdaki '8. Adım' kodunu tekrar çalıştırman gerekebilir.
def real_product_search(query: str) -> str:
    """
    Searches the real-world fashion database (CSV) for items.
    Example: 'Navy Blue Skirt', 'Red Heels'.
    """
    # df değişkeni global alandan gelir (8. Adımda tanımlamıştık)
    try:
        if 'df' not in globals() or df.empty:
            return "Ürün veritabanı (CSV) yüklü değil."
    except:
        return "Ürün veritabanı hatası."

    query = query.lower()
    print(f"\n🛍️ [MAĞAZA STOK ARAMASI]: '{query}'")

    keywords = query.split()
    results = df[df['search_text'].apply(lambda x: all(k in x for k in keywords))].head(5)

    if results.empty:
        return "Stoklarımızda bu kriterlere tam uyan ürün bulunamadı."

    output = f"📦 BULUNAN ÜRÜNLER (Toplam {len(results)} gösteriliyor):\n"
    for index, row in results.iterrows():
        output += f"- {row['productDisplayName']} ({row['season']}) - {row['usage']}\n"

    return output

# --- ARAÇLARI PAKETLEME ---
tools = [calculator, search_knowledge_base, real_product_search]
tools_with_desc = "\n\n".join([tool.__name__ + tool.__doc__ for tool in tools])
known_actions = dict([(tool.__name__, tool) for tool in tools])

print("✅ Araçlar Güncellendi: RAG Aracı artık ChromaDB kullanıyor!")

✅ Araçlar Güncellendi: RAG Aracı artık ChromaDB kullanıyor!


In [6]:
# Araçların düzgün tanımlanıp tanımlanmadığının kontrolü
print("--- AJANIN SAHİP OLDUĞU ARAÇLAR ---")
print(tools_with_desc)

--- AJANIN SAHİP OLDUĞU ARAÇLAR ---
calculator
    Calculates mathematical expressions.
    Useful for unit conversions (e.g. inch to cm) or budget calculations.
    

search_knowledge_base
    Use this tool to find stylistic rules, fabric details, and fashion theory.
    Searches the Vector Database (ChromaDB) for semantic matches from Guides and Wikipedia.
    

real_product_search
    Searches the real-world fashion database (CSV) for items.
    Example: 'Navy Blue Skirt', 'Red Heels'.
    


In [7]:
# Ajanın aksiyon listesini kontrol edelim
print("\n--- AKSİYON SÖZLÜĞÜ ---")
print(known_actions)


--- AKSİYON SÖZLÜĞÜ ---
{'calculator': <function calculator at 0x7e30cbf60ae0>, 'search_knowledge_base': <function search_knowledge_base at 0x7e30c9f52c00>, 'real_product_search': <function real_product_search at 0x7e3012054180>}


In [12]:
# 4. ADIM: AGENT SINIFI VE SYSTEM PROMPT

# --- SYSTEM PROMPT (Kişilik Tanımı) ---
# Ajanın nasıl davranacağını ve araçları nasıl kullanacağını burada anlatıyoruz.
system_prompt = f"""
You are a professional Personal Style Assistant.
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop, you output an Answer.

GUIDELINES:
1. You are an expert in Body Types, Color Analysis, Fabrics, and Dress Codes.
2. ALWAYS use 'search_knowledge_base' to retrieve specific styling rules.
3. NEVER make up styling rules. Check the knowledge base first.
4. Use 'calculator' for math operations (budget, unit conversion).
5. If the user asks about non-fashion topics (like stocks/politics), politely refuse.

Your available actions are:
{tools_with_desc}

Example Session:
Question: What fits a Pear body shape?
Thought: I need to check the body type guide for Pear shape.
Action: search_knowledge_base: Pear body shape rules
PAUSE
Observation: Wear A-line skirts and avoid hip pockets.
Answer: For a Pear shape, you should choose A-line skirts...
""".strip()

# --- AGENT CLASS (Beyin Yapısı) ---
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        # Groq API'ye isteği gönderiyoruz
        completion = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=self.messages,
            temperature=0,
            # Ajanın durup bizden araç çıktısı beklemesi için kritik durdurucular
            stop=["PAUSE", "Observation:", "<|"]
        )
        return completion.choices[0].message.content

print("✅ Ajan Sınıfı ve Prompt Hazırlandı!")

✅ Ajan Sınıfı ve Prompt Hazırlandı!


In [13]:
# MANUEL TEST ADIMI 1
# Ajanı başlatıyoruz
react_agent = Agent(system_prompt)

# Soruyu soruyoruz
question = "What fits a Pear body shape?"
result = react_agent(question)

print("--- AJANIN İLK TEPKİSİ ---")
print(result)

--- AJANIN İLK TEPKİSİ ---
Thought: I need to check the body type guide for Pear shape.
Action: search_knowledge_base: Pear body shape rules



In [14]:
# MANUEL TEST ADIMI 2
# Ajanın istediği aksiyonu (Action) biz manuel olarak yapıyoruz
# Not: Buradaki sorgu metnini, yukarıdaki çıktıdan kopyaladığımızı varsayıyoruz.
tool_result = search_knowledge_base("Pear body shape rules")

print("--- ARACIN DÖNDÜRDÜĞÜ SONUÇ (GÖZLEM) ---")
print(tool_result)


🔎 [CHROMADB ARAMASI]: 'Pear body shape rules'
--- ARACIN DÖNDÜRDÜĞÜ SONUÇ (GÖZLEM) ---
📚 Vektör Veritabanından Bulunanlar:
---
[Kaynak: SOURCE: Internal Guide (body_type_guide.txt)]
### The Pear (Triangle) Shape
- **Definition:** Hips are significantly wider than the shoulders. The waist is generally well-defined. This is the most common body shape.
- **Strategic Goal:** Draw attention to the upper body and elongate the legs to balance the wider hips.
- **Styling "Do's":**
  - Use boat necklines, statement necklaces, or scarves to draw the eye up.
  - Wear bright colors, prints, or detailed tops on the upper body.
  - Structured jackets with shoulder pads can visually widen the shoulders to match hip width.
  - A-line skirts and dresses that skim over the hips without clinging.
- **Styling "Don'ts":**
  - Avoid printed bottoms or pockets on the hips, as they add volume.
  - Avoid skinny jeans that bunch at the ankles; bootcut or straight-leg pants are better.
  - Hemlines should not e

In [15]:
# MANUEL TEST ADIMI 3
# Sonucu ajana geri besliyoruz
next_prompt = "Observation: {}".format(tool_result)

print("--- AJANA GÖNDERİLEN GÖZLEM ---")
print(next_prompt)

# Ajanın son cevabını alıyoruz
final_answer = react_agent(next_prompt)

print("\n--- NİHAİ CEVAP ---")
print(final_answer)

--- AJANA GÖNDERİLEN GÖZLEM ---
Observation: 📚 Vektör Veritabanından Bulunanlar:
---
[Kaynak: SOURCE: Internal Guide (body_type_guide.txt)]
### The Pear (Triangle) Shape
- **Definition:** Hips are significantly wider than the shoulders. The waist is generally well-defined. This is the most common body shape.
- **Strategic Goal:** Draw attention to the upper body and elongate the legs to balance the wider hips.
- **Styling "Do's":**
  - Use boat necklines, statement necklaces, or scarves to draw the eye up.
  - Wear bright colors, prints, or detailed tops on the upper body.
  - Structured jackets with shoulder pads can visually widen the shoulders to match hip width.
  - A-line skirts and dresses that skim over the hips without clinging.
- **Styling "Don'ts":**
  - Avoid printed bottoms or pockets on the hips, as they add volume.
  - Avoid skinny jeans that bunch at the ankles; bootcut or straight-leg pants are better.
  - Hemlines should not end at the widest part of the hips.
---
[Kay

In [16]:
# 5. ADIM: OTOMATİK DÖNGÜ (LOOR)
import re

# Regex: Ajanın "Action:" ile başlayan satırlarını yakalar
action_re = re.compile(r'^Action: (\w+): (.*)$')

def query(question, max_turns=10):
    i = 0
    # Ajanı başlatıyoruz
    bot = Agent(system_prompt)
    next_prompt = question

    print(f"\n{'='*40}\nSORU: {question}\n{'='*40}")

    while i < max_turns:
        i += 1
        # 1. Ajanı çalıştır (Cevabı al)
        result = bot(next_prompt)

        # Ajanın ne düşündüğünü mavi renkte ekrana yaz
        print(f"\033[94m{result}\033[0m")

        # 2. İçinde "Action" var mı diye bak (Regex ile)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]

        if actions:
            # Aksiyon varsa, hangi araç ve hangi girdi olduğunu bul
            action, action_input = actions[0].groups()

            if action not in known_actions:
                print(f"⚠️ HATA: Bilinmeyen Aksiyon -> {action}")
                next_prompt = "Observation: I don't know this tool."
                continue

            # 3. Aracı (Tool) Gerçekten Çalıştır
            print(f"⚙️ ÇALIŞTIRILIYOR: {action} -> {action_input}")

            try:
                observation = known_actions[action](action_input)
            except Exception as e:
                observation = f"Error: {e}"

            print(f"📝 GÖZLEM: {observation}")

            # 4. Sonucu ajana geri gönder (Döngü başa döner)
            next_prompt = f"Observation: {observation}"

        else:
            # Aksiyon yoksa, cevap gelmiş demektir.
            if "Answer:" in result:
                print(f"\n✅ NİHAİ CEVAP:\n{result.split('Answer:')[-1].strip()}")
            return

print("✅ Döngü mekanizması hazır! Artık asistanı test edebiliriz.")

✅ Döngü mekanizması hazır! Artık asistanı test edebiliriz.


In [17]:
# 6. ADIM: TEST SENARYOLARI (PDF Yönergesine Tam Uygun)

print("\n-----------------------------------------------------------")
print("SENARYO A: Tek Atımlık RAG Sorgusu (Doğrudan Bilgi)")
print("Amaç: PDF Madde 4.1 - Doğrudan dökümandan bulunacak cevaplar.")
print("-----------------------------------------------------------")

# Bu sorgu sadece RAG aracını kullanmalı.
query("What are the specific styling rules for a Pear body shape?")


print("\n-----------------------------------------------------------")
print("SENARYO B: Multi-Hop (Çok Adımlı) Sorgu")
print("Amaç: PDF Madde 4.2 - Önce dökümandan bilgi çekip, sonra işlem yapma.")
print("-----------------------------------------------------------")

# BURASI ÇOK ÖNEMLİ:
# Ajanın önce RAG ile "Mini etek boyu" bilgisini bulmasını (Örn: 15 inch),
# SONRA calculator ile bunu cm'ye çevirmesini istiyoruz.
# (Not: Eğer dosyalarda spesifik bir sayı yoksa, ajan genel bir çeviri yapmaya çalışacaktır)

complex_query = """
According to the body type guide, short skirts are often mentioned.
Assume the ideal mini skirt length is described as 16 inches in the fashion industry.
Use the calculator to convert this 16 inches into centimeters to give me the metric length.
"""

query(complex_query)


-----------------------------------------------------------
SENARYO A: Tek Atımlık RAG Sorgusu (Doğrudan Bilgi)
Amaç: PDF Madde 4.1 - Doğrudan dökümandan bulunacak cevaplar.
-----------------------------------------------------------

SORU: What are the specific styling rules for a Pear body shape?
Thought: I need to check the body type guide for Pear shape to provide accurate styling rules.

Action: search_knowledge_base: Pear body shape rules


⚙️ ÇALIŞTIRILIYOR: search_knowledge_base -> Pear body shape rules

🔎 [CHROMADB ARAMASI]: 'Pear body shape rules'
📝 GÖZLEM: 📚 Vektör Veritabanından Bulunanlar:
---
[Kaynak: SOURCE: Internal Guide (body_type_guide.txt)]
### The Pear (Triangle) Shape
- **Definition:** Hips are significantly wider than the shoulders. The waist is generally well-defined. This is the most common body shape.
- **Strategic Goal:** Draw attention to the upper body and elongate the legs to balance the wider hips.
- **Styling "Do's":**
  - Use boat necklines, statement 

In [18]:
# =============================================================================
# 7. ADIM: AGENT BENCHMARK SUITE (Ajan Performans Testi)
# Amaç: Ajanın doğru aracı seçip seçmediğini ve beklenen cevapları verip vermediğini puanlamak.
# =============================================================================

import time

# 1. TEST VERİ SETİ (GOLDEN DATASET)
# Soruları, beklenen aracı (Tool) ve cevapta geçmesi gereken anahtar kelimeleri tanımlıyoruz.
benchmark_dataset = [
    {
        "question": "Convert 100 inches to centimeters.",
        "expected_tool": "calculator",
        "keywords": ["254", "cm"],
        "category": "Math"
    },
    {
        "question": "What fits a Pear body shape?",
        "expected_tool": "search_knowledge_base",
        "keywords": ["A-line", "hips", "shoulders"],
        "category": "RAG"
    },
    {
        "question": "I have cool undertones and blue veins. What colors suit me?",
        "expected_tool": "search_knowledge_base",
        "keywords": ["Winter", "Summer", "Jewel", "Blue"],
        "category": "Reasoning"
    },
    {
        "question": "Calculate 500 divided by 20.",
        "expected_tool": "calculator",
        "keywords": ["25"],
        "category": "Math"
    },
    {
        "question": "What fabric is best for summer and breathable?",
        "expected_tool": "search_knowledge_base", # Veya product_search varsa o
        "keywords": ["Cotton", "Linen", "Silk"],
        "category": "RAG"
    }
]

# 2. BENCHMARK FONKSİYONU
def run_benchmark(agent_fn, dataset):
    results = []
    print(f"🚀 BENCHMARK BAŞLIYOR... ({len(dataset)} Soru)\n")

    total_score = 0

    for i, item in enumerate(dataset):
        print(f"🔹 Soru {i+1}: {item['question']}")

        # Ajanı çalıştır (Output'u yakalamak için query fonksiyonunu biraz simüle ediyoruz)
        # Not: query fonksiyonun print yaptığı için burada outputu analiz etmek adına
        # query fonksiyonunun mantığını simple_agent_call ile taklit ediyoruz veya
        # query fonksiyonunu 'return string' yapacak şekilde güncellediysen onu kullanırız.
        # Biz burada output'u yakalamak için Agent sınıfını doğrudan çağıracağız.

        bot = Agent(system_prompt)
        response = bot(item['question']) # İlk adım (Thought + Action)

        # --- DEĞERLENDİRME ---
        score = 0
        status = "❌ BAŞARISIZ"
        detected_tool = "None"

        # 1. Kriter: Doğru Tool Seçildi mi?
        if f"Action: {item['expected_tool']}" in response:
            score += 1
            detected_tool = item['expected_tool']

            # Aracı gerçekten çalıştırıp nihai cevabı alalım (Keywords kontrolü için)
            # Basit regex ile inputu alalım
            import re
            match = re.search(r'Action: \w+: (.*)', response)
            if match:
                action_input = match.group(1)
                try:
                    observation = known_actions[item['expected_tool']](action_input)
                    final_prompt = f"Observation: {observation}"
                    final_answer = bot(final_prompt) # Nihai cevap

                    # 2. Kriter: Anahtar Kelimeler Var mı?
                    missing_keywords = [k for k in item['keywords'] if k.lower() not in final_answer.lower() and k.lower() not in observation.lower()]
                    if not missing_keywords: # Hepsi varsa veya çoğu varsa
                         score += 1 # Tam puan
                    else:
                        print(f"   ⚠️ Eksik Kelimeler: {missing_keywords}")

                except:
                    pass

        # Puanlama (Maksimum 2 puan: 1 Tool Doğruluğu, 1 İçerik Doğruluğu)
        if score >= 1: status = "✅ BAŞARILI" if score == 2 else "⚠️ KISMEN"

        print(f"   Durum: {status} | Beklenen Tool: {item['expected_tool']} | Algılanan: {detected_tool}")
        print("-" * 50)

        results.append({
            "question": item['question'],
            "score": score,
            "max_score": 2
        })
        total_score += score

    # 3. RAPORLAMA
    max_total = len(dataset) * 2
    accuracy = (total_score / max_total) * 100

    print(f"\n🏆 BENCHMARK SONUCU:")
    print(f"Toplam Puan: {total_score}/{max_total}")
    print(f"Başarı Oranı: %{accuracy:.2f}")

    return results

# Benchmark'ı Çalıştır
benchmark_results = run_benchmark(Agent, benchmark_dataset)

🚀 BENCHMARK BAŞLIYOR... (5 Soru)

🔹 Soru 1: Convert 100 inches to centimeters.
   Durum: ⚠️ KISMEN | Beklenen Tool: calculator | Algılanan: calculator
--------------------------------------------------
🔹 Soru 2: What fits a Pear body shape?

🔎 [CHROMADB ARAMASI]: 'Pear body shape rules'
   Durum: ✅ BAŞARILI | Beklenen Tool: search_knowledge_base | Algılanan: search_knowledge_base
--------------------------------------------------
🔹 Soru 3: I have cool undertones and blue veins. What colors suit me?

🔎 [CHROMADB ARAMASI]: 'color analysis cool undertones blue veins'
   ⚠️ Eksik Kelimeler: ['Summer']
   Durum: ⚠️ KISMEN | Beklenen Tool: search_knowledge_base | Algılanan: search_knowledge_base
--------------------------------------------------
🔹 Soru 4: Calculate 500 divided by 20.
   Durum: ⚠️ KISMEN | Beklenen Tool: calculator | Algılanan: calculator
--------------------------------------------------
🔹 Soru 5: What fabric is best for summer and breathable?

🔎 [CHROMADB ARAMASI]: 'breatha

In [25]:
# =============================================================================
# 8. ADIM: REAL-WORLD DATA ENTEGRASYONU VE DİL DESTEKLİ PROMPT
# =============================================================================

import pandas as pd

# 1. KAGGLE VERİ SETİNİ YÜKLEME VE TEMİZLEME
print("📂 Gerçek Veri Seti (styles.csv) okunuyor...")

try:
    # on_bad_lines='skip': Bozuk satırları atla (Gerçek verilerde hep olur)
    df = pd.read_csv('styles.csv', on_bad_lines='skip')

    # Sütun isimlerini küçük harfe çevirelim ki hata yapmayalım
    df.columns = [c.strip() for c in df.columns]

    # İşimize yarayacak sütunları seçelim ve temizleyelim
    df = df[['id', 'gender', 'masterCategory', 'subCategory', 'articleType', 'baseColour', 'season', 'usage', 'productDisplayName']]

    # NaN (Boş) değerleri temizle
    df = df.dropna()

    # Metinleri küçük harfe çevir (Arama kolaylığı için)
    # Bu sütun, Ajanın arama yapacağı ana havuzdur.
    df['search_text'] = df['productDisplayName'].str.lower() + " " + \
                        df['articleType'].str.lower() + " " + \
                        df['baseColour'].str.lower() + " " + \
                        df['usage'].str.lower()

    print(f"✅ VERİ SETİ BAŞARIYLA YÜKLENDİ! Toplam Ürün Sayısı: {len(df)}")
    print("Örnek Veri:")
    print(df[['productDisplayName', 'baseColour', 'season']].head(3))

except FileNotFoundError:
    print("❌ HATA: 'styles.csv' dosyası bulunamadı! Lütfen Colab'e yüklediğinden emin ol.")
    df = pd.DataFrame()


# 2. GELİŞMİŞ ÜRÜN ARAMA ARACI (GERÇEK VERİ İÇİN)
def real_product_search(query: str) -> str:
    """
    Searches the HUGE real-world fashion database.
    Query should focus on Item Type (Shirt, Shoes), Color, or Usage (Casual, Formal).
    Do NOT query body types directly here. Query concrete items.
    Example: 'Navy Blue Skirt', 'Red Heels', 'Casual Shirt'
    """
    if df.empty:
        return "Veri seti yüklü değil."

    query = query.lower()
    print(f"\n🛍️ [MAĞAZA STOK ARAMASI]: '{query}'")

    # Arama kelimelerini parçala
    keywords = query.split()

    # Pandas filtreleme (Tüm kelimeler search_text içinde geçmeli)
    results = df[df['search_text'].apply(lambda x: all(k in x for k in keywords))].head(5)

    if results.empty:
        return "Stoklarımızda bu kriterlere tam uyan ürün bulunamadı. Lütfen daha genel arayın (Örn: 'Blue Skirt')."

    output = f"📦 BULUNAN ÜRÜNLER (Toplam {len(results)} gösteriliyor):\n"
    for index, row in results.iterrows():
        output += f"- {row['productDisplayName']} ({row['season']} Season) - {row['usage']}\n"

    return output


# 3. ARAÇLARI PAKETLEME (Garanti olsun diye tekrar tanımlıyoruz)
# Not: calculator ve search_knowledge_base önceki adımlardan geliyor.
tools = [calculator, search_knowledge_base, real_product_search]
tools_with_desc = "\n\n".join([tool.__name__ + tool.__doc__ for tool in tools])
known_actions = dict([(tool.__name__, tool) for tool in tools])


# 4. SYSTEM PROMPT GÜNCELLEMESİ (DİL DESTEĞİ EKLENDİ)
# Burası en kritik kısım. Ajanın beynine "Tercüman" kurallarını yazıyoruz.
system_prompt = f"""
You are an expert Personal Stylist powered by a Real-World Fashion Database.
You operate in a loop of Thought, Action, PAUSE, Observation.

CRITICAL LANGUAGE RULE:
- The Knowledge Base and Product Database are in ENGLISH.
- If the user asks in TURKISH (or any other language):
  1. THINK in English (Plan your steps in English).
  2. TRANSLATE the search queries to English before using tools (e.g., search for 'Blue Skirt', NOT 'Mavi Etek').
  3. ANSWER in the user's original language (Turkish).

CRITICAL RULES:
1. DO NOT narrative or talk to the user during the process. Just think and act.
2. YOU MUST USE THIS EXACT FORMAT for actions:
   Thought: [Your reasoning in English]
   Action: [Tool Name]: [Search Query in English]
   PAUSE

3. Available Tools:
{tools_with_desc}

4. STRATEGY:
   - STEP 1: Search styling rules using 'search_knowledge_base'.
   - STEP 2: Search actual products using 'real_product_search'.
   - STEP 3: Combine them in the final Answer.

Example Session (Turkish User):
Question: Mavi etek arıyorum.
Thought: User wants a blue skirt. Database is English. I will search for 'Blue Skirt'.
Action: real_product_search: Blue Skirt
PAUSE
Observation: Found H&M Blue Skirt...
Answer: İstediğiniz kriterlere uygun H&M marka mavi eteği buldum...
""".strip()

print("✅ Ajan Başarıyla Güncellendi!")
print("   - Veri Seti: Kaggle (44k+ Ürün)")
print("   - Özellik: Türkçe Soruları İngilizce Veriyle Yanıtlayabiliyor (Auto-Translation)")

📂 Gerçek Veri Seti (styles.csv) okunuyor...
✅ VERİ SETİ BAŞARIYLA YÜKLENDİ! Toplam Ürün Sayısı: 44077
Örnek Veri:
                   productDisplayName baseColour  season
0    Turtle Check Men Navy Blue Shirt  Navy Blue    Fall
1  Peter England Men Party Blue Jeans       Blue  Summer
2            Titan Women Silver Watch     Silver  Winter
✅ Ajan Başarıyla Güncellendi!
   - Veri Seti: Kaggle (44k+ Ürün)
   - Özellik: Türkçe Soruları İngilizce Veriyle Yanıtlayabiliyor (Auto-Translation)


In [26]:
# TEST SENARYOSU
# Ajanın "Armut Tipi" -> "A-line" veya "Bootcut" bağlantısını kurup
# Gerçek veri setinden marka/model getirmesini bekliyoruz.

query("I have a Pear body shape and I need an outfit for a Casual summer day. Suggest me items available in the store.")


SORU: I have a Pear body shape and I need an outfit for a Casual summer day. Suggest me items available in the store.
Thought: The user has a Pear body shape and needs a casual summer outfit. For Pear body shapes, it's recommended to balance the hips and thighs with tops and jackets that add volume to the upper body, and to draw attention away from the lower body with darker colors or patterns. I will search for tops that can create a balanced look.

Action: search_knowledge_base: Pear body shape styling tips

⚙️ ÇALIŞTIRILIYOR: search_knowledge_base -> Pear body shape styling tips

🔎 [CHROMADB ARAMASI]: 'Pear body shape styling tips'
📝 GÖZLEM: 📚 Vektör Veritabanından Bulunanlar:
---
[Kaynak: SOURCE: Internal Guide (body_type_guide.txt)]
### The Pear (Triangle) Shape
- **Definition:** Hips are significantly wider than the shoulders. The waist is generally well-defined. This is the most common body shape.
- **Strategic Goal:** Draw attention to the upper body and elongate the legs to b

In [28]:
# =============================================================================
# DÜZELTME: DAHA SIKI (STRICT) SYSTEM PROMPT
# =============================================================================

# Araçları tekrar tanımlayalım ki garanti olsun
tools = [calculator, search_knowledge_base, real_product_search]
tools_with_desc = "\n\n".join([tool.__name__ + tool.__doc__ for tool in tools])
known_actions = dict([(tool.__name__, tool) for tool in tools])

# YENİ VE SERT SYSTEM PROMPT
system_prompt = f"""
You are an expert Personal Stylist powered by a Real-World Fashion Database.
You operate in a loop of Thought, Action, PAUSE, Observation.

CRITICAL RULES:
1. DO NOT narrative or talk to the user during the process. Just think and act.
2. YOU MUST USE THIS EXACT FORMAT for actions:
   Thought: [Your reasoning]
   Action: [Tool Name]: [Search Query]
   PAUSE

3. Available Tools:
{tools_with_desc}

4. STRATEGY:
   - STEP 1: Search styling rules using 'search_knowledge_base'.
   - STEP 2: Search actual products using 'real_product_search'.
   - STEP 3: Combine them in the final Answer.

Example Session:
Question: I need a summer outfit.
Thought: I should look for summer fabrics first.
Action: search_knowledge_base: summer fabrics
PAUSE
Observation: Cotton and Linen are best.
Thought: Now I will search for Cotton Shirts in the store.
Action: real_product_search: Cotton Shirt
PAUSE
Observation: ...list of products...
Answer: I recommend these cotton shirts...
""".strip()

print("✅ Ajanın beyni güncellendi: Artık daha disiplinli!")

✅ Ajanın beyni güncellendi: Artık daha disiplinli!


In [31]:
print("\n🔥 FİNAL SENARYOSU (Düzeltilmiş):")
query("I have a Pear body shape and I need an outfit for a Casual summer day. Suggest me items available in the store.")


🔥 FİNAL SENARYOSU (Düzeltilmiş):

SORU: I have a Pear body shape and I need an outfit for a Casual summer day. Suggest me items available in the store.
Thought: As a Pear body shape, the individual carries weight in the hips and thighs, so I should look for styling rules that balance the silhouette, likely involving A-line dresses or skirts, and tops that draw attention to the upper body.

Action: search_knowledge_base: pear body shape styling tips

⚙️ ÇALIŞTIRILIYOR: search_knowledge_base -> pear body shape styling tips

🔎 [CHROMADB ARAMASI]: 'pear body shape styling tips'
📝 GÖZLEM: 📚 Vektör Veritabanından Bulunanlar:
---
[Kaynak: SOURCE: Internal Guide (body_type_guide.txt)]
### The Pear (Triangle) Shape
- **Definition:** Hips are significantly wider than the shoulders. The waist is generally well-defined. This is the most common body shape.
- **Strategic Goal:** Draw attention to the upper body and elongate the legs to balance the wider hips.
- **Styling "Do's":**
  - Use boat neck

In [32]:
# =============================================================================
# 9. ADIM: GENİŞLETİLMİŞ BENCHMARK SUITE (TÜRKÇE + İNGİLİZCE KARIŞIK)
# =============================================================================
import re

# 1. ALTIN TEST VERİ SETİ (11 Çeşitli Senaryo)
# Hem Türkçe dil desteğini hem de orijinal 10 senaryoyu içerir.
advanced_benchmark_data = [
    # --- YENİ EKLENEN: TÜRKÇE DİL TESTİ (Cross-Lingual Capability) ---
    {
        "scenario": "Turkish Language Query (Cross-Lingual Search)",
        "question": "Armut vücut tipi için yazlık kıyafet önerir misin?",
        "expected_tools": ["search_knowledge_base", "real_product_search"],
        "min_keywords": ["etek", "elbise", "öner", "buldum"], # Cevabın Türkçe olması beklenir
        "difficulty": "Hard"
    },

    # --- KATEGORİ 1: KARMAŞIK MUHAKEME (RAG + CSV) ---
    {
        "scenario": "Pear Shape + Summer Outfit",
        "question": "I have a Pear body shape. Find me a suitable outfit for Summer from the store.",
        "expected_tools": ["search_knowledge_base", "real_product_search"],
        "min_keywords": ["found", "skirt", "dress", "top"],
        "difficulty": "Hard"
    },
    {
        "scenario": "Apple Shape + Formal Wear",
        "question": "What should an Apple body shape wear for a formal event? Check the inventory.",
        "expected_tools": ["search_knowledge_base", "real_product_search"],
        "min_keywords": ["found", "trousers", "jacket", "formal"],
        "difficulty": "Hard"
    },
    {
        "scenario": "Winter Fabric Recommendation",
        "question": "Which fabric is best for Winter? After finding the rule, search for products with that fabric.",
        "expected_tools": ["search_knowledge_base", "real_product_search"],
        "min_keywords": ["found", "wool", "coat", "jacket"],
        "difficulty": "Hard"
    },

    # --- KATEGORİ 2: DOĞRUDAN ÜRÜN ARAMA (CSV ONLY) ---
    {
        "scenario": "Specific Item Search (Red Dress)",
        "question": "Do you have any Red Dress in the stock?",
        "expected_tools": ["real_product_search"],
        "min_keywords": ["found", "Red", "Dress"],
        "difficulty": "Medium"
    },
    {
        "scenario": "Category Search (Heels)",
        "question": "Show me available Heels.",
        "expected_tools": ["real_product_search"],
        "min_keywords": ["found", "Heels"],
        "difficulty": "Medium"
    },
    {
        "scenario": "Usage Search (Casual)",
        "question": "I need some Casual clothes. What do you have?",
        "expected_tools": ["real_product_search"],
        "min_keywords": ["found", "Casual"],
        "difficulty": "Medium"
    },

    # --- KATEGORİ 3: BİLGİ VE KURAL SORGUSU (RAG ONLY) ---
    {
        "scenario": "Color Theory (Cool Undertone)",
        "question": "I have cool undertones. What colors should I wear?",
        "expected_tools": ["search_knowledge_base"],
        "min_keywords": ["blue", "purple", "winter", "summer"],
        "difficulty": "Medium"
    },
    {
        "scenario": "Styling Don'ts (Inverted Triangle)",
        "question": "What should an Inverted Triangle body shape avoid wearing?",
        "expected_tools": ["search_knowledge_base"],
        "min_keywords": ["shoulder", "neckline", "avoid"],
        "difficulty": "Medium"
    },

    # --- KATEGORİ 4: MATEMATİKSEL İŞLEMLER (CALCULATOR) ---
    {
        "scenario": "Unit Conversion",
        "question": "Convert 30 inches to centimeters using the calculator.",
        "expected_tools": ["calculator"],
        "min_keywords": ["76", "2"],
        "difficulty": "Easy"
    },
    {
        "scenario": "Budget Calculation",
        "question": "If I buy a dress for 50 dollars and shoes for 30 dollars, what is the total?",
        "expected_tools": ["calculator"],
        "min_keywords": ["80"],
        "difficulty": "Easy"
    }
]

# 2. BENCHMARK MOTORU
def run_advanced_benchmark(dataset):
    print(f"🚀 STRES TESTİ BAŞLIYOR... ({len(dataset)} Senaryo)\n")

    total_score = 0
    max_total_score = len(dataset) * 100 # Her soru 100 puan
    action_re = re.compile(r'^Action: (\w+): (.*)$')

    results_table = []

    for i, case in enumerate(dataset):
        print(f"🔹 Senaryo {i+1}: {case['scenario']}")

        # Ajanı Sıfırla
        bot = Agent(system_prompt)
        next_prompt = case['question']
        used_tools = set()
        conversation_log = ""

        # Döngüyü Çalıştır (Max 5 adım)
        for turn in range(5):
            result = bot(next_prompt)
            conversation_log += result + "\n"

            actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]

            if actions:
                action, action_input = actions[0].groups()
                used_tools.add(action)

                if action in known_actions:
                    try:
                        obs = known_actions[action](action_input)
                    except:
                        obs = "Error"
                    next_prompt = f"Observation: {obs}"
                else:
                    next_prompt = "Observation: Tool not found"
            else:
                if "Answer:" in result:
                    break

        # --- PUANLAMA ---
        score = 0
        # Beklenen araçların kullanımı kontrol ediliyor
        missing_tools = [t for t in case['expected_tools'] if t not in used_tools]

        # Kriter 1: Araç Kullanımı (50 Puan)
        if not missing_tools:
            score += 50

        # Kriter 2: Anahtar Kelime Yakalama (50 Puan)
        # Cevabın içinde beklenen kelimeler (Türkçe veya İngilizce) var mı?
        found_keywords = [k for k in case['min_keywords'] if k.lower() in conversation_log.lower()]
        if len(found_keywords) >= 1:
            score += 50

        total_score += score

        # Sonucu kaydet
        status = "✅" if score == 100 else "⚠️" if score > 0 else "❌"
        print(f"   Sonuç: {status} ({score}/100) | Eksik Araç: {missing_tools} | Bulunan Kelimeler: {len(found_keywords)}")
        print("-" * 50)

        results_table.append({
            "Scenario": case['scenario'],
            "Difficulty": case['difficulty'],
            "Score": score
        })

    # 3. FİNAL RAPORU
    accuracy = (total_score / max_total_score) * 100
    print(f"\n🏆 GENEL PERFORMANS:")
    print(f"Toplam Puan: {total_score}/{max_total_score}")
    print(f"Başarı Oranı: %{accuracy:.1f}")

    # Rapor için özet tabloyu yazdır (Markdown formatında)
    print("\n📋 RAPOR İÇİN TABLO FORMATI:")
    print("| Senaryo | Zorluk | Puan |")
    print("|---|---|---|")
    for r in results_table:
        print(f"| {r['Scenario']} | {r['Difficulty']} | {r['Score']} |")

# Testi Başlat
run_advanced_benchmark(advanced_benchmark_data)

🚀 STRES TESTİ BAŞLIYOR... (11 Senaryo)

🔹 Senaryo 1: Turkish Language Query (Cross-Lingual Search)

🔎 [CHROMADB ARAMASI]: 'pear shaped body summer outfit'

🛍️ [MAĞAZA STOK ARAMASI]: 'boat neck tops, statement necklaces, a-line skirts, bright colored tops'

🛍️ [MAĞAZA STOK ARAMASI]: 'a-line skirt, bright tops'

🛍️ [MAĞAZA STOK ARAMASI]: 'colorful tops, wide leg pants'

🛍️ [MAĞAZA STOK ARAMASI]: 'summer tops'
   Sonuç: ✅ (100/100) | Eksik Araç: [] | Bulunan Kelimeler: 2
--------------------------------------------------
🔹 Senaryo 2: Pear Shape + Summer Outfit

🔎 [CHROMADB ARAMASI]: 'summer outfit for Pear body shape'

🛍️ [MAĞAZA STOK ARAMASI]: 'bright tops with a-line skirts'

🛍️ [MAĞAZA STOK ARAMASI]: 'bright summer tops'

🛍️ [MAĞAZA STOK ARAMASI]: 'white blouse'

🛍️ [MAĞAZA STOK ARAMASI]: 'a-line skirt'
   Sonuç: ✅ (100/100) | Eksik Araç: [] | Bulunan Kelimeler: 4
--------------------------------------------------
🔹 Senaryo 3: Apple Shape + Formal Wear

🔎 [CHROMADB ARAMASI]: 'Apple bod